In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
DEBUG=False

In [ ]:
%run ./nb_core/root_dirs.ipynb
setup_syspath_disentangle(DEBUG)
%run ./nb_core/disentangle_imports.ipynb

In [ ]:
nmodel_dir = "/home/ashesh.ashesh/training/noise_model/2509/8"
# nmodel_dir = '/home/ashesh.ashesh/training/noise_model/2402/61'

histnoisemodel_fpath = None
gmmnoisemodel_fpath = None
for fname in os.listdir(nmodel_dir):
    if fname.startswith('HistNoiseModel'):
        assert histnoisemodel_fpath is None
        histnoisemodel_fpath = os.path.join(nmodel_dir, fname)
    elif fname.startswith('GMMNoiseModel'):
        assert gmmnoisemodel_fpath is None
        gmmnoisemodel_fpath = os.path.join(nmodel_dir, fname)
print(gmmnoisemodel_fpath)
print(histnoisemodel_fpath)

In [ ]:
from disentangle.utils import plotProbabilityDistribution
from disentangle.nets.gmm_noise_model import GaussianMixtureNoiseModel
from disentangle.nets.hist_noise_model import HistNoiseModel


nmodel_params = np.load(gmmnoisemodel_fpath)
gmm_model = GaussianMixtureNoiseModel(params=nmodel_params)
histdata = np.load(histnoisemodel_fpath)
hist_model = HistNoiseModel(histdata)
bins = histdata.shape[-1]

# plotting
signalBinIndex= 20
data_dict = plotProbabilityDistribution(signalBinIndex=signalBinIndex, 
                            histogramNoiseModel=hist_model,
                            gaussianMixtureNoiseModel=gmm_model,
                            device='cpu')
# data_dict['gmm']['x'][data_dict['gmm']['p'].argmax()]

In [ ]:
params = gmm_model.getGaussianParameters(signalBinIndex)
np.sqrt(np.sum((np.array(params[-6:])) * np.array(params[6:12])**2))

In [ ]:
import torch
x = torch.linspace(100, 400, 100).unsqueeze(1)
output = gmm_model.getGaussianParameters(x)

In [ ]:
data = torch.stack(output[-6:], dim=-1).squeeze()

In [ ]:
torch.sum(data, dim=-1)

In [ ]:
import matplotlib.pyplot as plt
from disentangle.configs.nikola_denoisplit import get_config
from disentangle.data_loader.nikola_7D_rawdata_loader import get_train_val_data, DataSplitType, NikolaChannelList

config = get_config()
config.data.enable_gaussian_noise = False
config.data.dset_type_per_channel = None
config.data.dset_type = '500ms'
config.data.channel_idx_list = [NikolaChannelList.Ch_A, NikolaChannelList.Ch_B,  NikolaChannelList.Ch_AB]
config.data.num_channels = len(config.data.channel_idx_list)
# config.data.dset_type_per_channel = ['5ms', '5ms', '500ms']

datadir = '/group/jug/ashesh/data/nikola_data/20240531/'
data500 = get_train_val_data(datadir, config.data, DataSplitType.Train,
                            config.training.val_fraction, config.training.test_fraction)
_,ax = plt.subplots(figsize=(18,6),ncols=3)
ax[0].imshow(data500[0,...,0])
ax[1].imshow(data500[0,...,1])
ax[2].imshow(data500[0,...,2])
# 'high', [0, 1]

In [ ]:
import matplotlib.pyplot as plt
from disentangle.configs.nikola_denoisplit import get_config
from disentangle.data_loader.nikola_7D_rawdata_loader import get_train_val_data, DataSplitType, NikolaChannelList

config = get_config()
config.data.enable_gaussian_noise = False
config.data.dset_type_per_channel = None
config.data.dset_type = '3ms'
config.data.channel_idx_list = [NikolaChannelList.Ch_A, NikolaChannelList.Ch_B,  NikolaChannelList.Ch_AB]
config.data.num_channels = len(config.data.channel_idx_list)
# config.data.dset_type_per_channel = ['5ms', '5ms', '500ms']

datadir = '/group/jug/ashesh/data/nikola_data/20240531/'
data5 = get_train_val_data(datadir, config.data, DataSplitType.Train,
                            config.training.val_fraction, config.training.test_fraction)
_,ax = plt.subplots(figsize=(18,6),ncols=3)
ax[0].imshow(data5[0,...,0])
ax[1].imshow(data5[0,...,1])
ax[2].imshow(data5[0,...,2])
# 'high', [0, 1]

In [ ]:
input5_raw = data5[...,-1].copy()
input500_raw = data500[...,-1].copy()

input5 = input5_raw.copy()
input500 = input500_raw.copy()

_,ax = plt.subplots(figsize=(12,6),ncols=2)
ax[0].imshow(input5[0,-200:,-200:])
ax[1].imshow(input500[0,-200:,-200:])

q500 = np.quantile(input500, [0.01, 0.99])
q5 = np.quantile(input5, [0.01, 0.99])
print(q5, q500)

input5[input5 < q5[0]] = q5[0]
input500[input500 < q500[0]] = q500[0]

input5 -= q5[0]
input500 -= q500[0]

factor = input5.max() / input500.max()
input500 *= factor
input500 += q5[0]


q500_normalized = np.quantile(input500, [0.01, 0.99])
q5_normalized = np.quantile(input5, [0.01, 0.99])
print(q5_normalized, q500_normalized)



In [ ]:
noisy_output = gmm_model.sample(torch.Tensor(input500[0,-200:,-200:].flatten()))
noisy_output = noisy_output.reshape(200,200).numpy()
_,ax = plt.subplots(figsize=(18,6),ncols=3)
ax[0].imshow(input5_raw[0,-200:,-200:], vmin = q5[0], vmax = q5[1]) 
ax[1].imshow(input500[0,-200:,-200:])
ax[2].imshow(noisy_output, vmin = q5[0], vmax = q5[1])            

In [ ]:
import numpy as np
import cv2
import glob

def estimate_poisson_gaussian_parameters(image_data, num_bins=5):
    means = []
    variances = []
    
    quantiles = np.arange(0.01, 0.99, 0.98/num_bins).tolist()
    quantiles = quantiles + [1]
    quantiles[0] = 0
    assert len(quantiles)-1 == num_bins, f'Expected {num_bins+1} quantiles, got {len(quantiles)}'

    for img in image_data:
        # img = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE).astype(np.float32)
        # Group pixels by intensity bins (here we take each pixel)
        # Alternative: use patches or quantize intensities
        pixels = img.flatten()
        qvals = np.quantile(pixels, quantiles)
        minv= None
        for eidx in range(1,len(qvals)):
            if minv is None:
                minv = qvals[eidx-1]
            maxv = qvals[eidx]
            if maxv <= minv + 5:
                print(f'Skipping bin {minv} to {maxv} as too small')
                continue
            print(minv, maxv)
            mask = (pixels >= minv) & (pixels < maxv)
            px_group = pixels[mask]
            print(minv, maxv, px_group.min(), px_group.max(), len(px_group))
            if len(px_group) > 1:
                means.append(px_group.mean())
                variances.append(np.var(px_group))
            minv = None

    means = np.array(means)
    variances = np.array(variances)
    
    # Fit linear model Variance = alpha * Mean + beta
    A = np.stack([means, np.ones_like(means)], axis=1)
    alpha, beta = np.linalg.lstsq(A, variances, rcond=None)[0]
    
    return alpha, beta, means, variances

# Example usage:

alpha, beta, mean, variances = estimate_poisson_gaussian_parameters([input5_raw.astype(np.uint16)])

print(f"Estimated Poisson noise parameter (alpha): {alpha}")
print(f"Estimated Gaussian noise variance (beta): {beta}")


In [ ]:
mean, variances

In [ ]:
np.quantile(input5_raw, [0.01, 0.99])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(mean, variances)

In [ ]:
input5_raw.max()

In [ ]:
from disentangle.core.tiff_reader import load_tiff
import matplotlib.pyplot as plt

data = load_tiff('/facility/imganfacusers/Ashesh/Neurons_data/Composite/Composite_007.tif')
seg = load_tiff('/facility/imganfacusers/Ashesh/Neurons_data/Masks/disentangled_007.tif')
data.shape, seg.shape

In [ ]:
_,ax = plt.subplots(figsize=(15,9),ncols=5, nrows=3)
ax = ax.reshape(-1)
print(ax.shape)
for i in range(15):
    ax[i].imshow(seg[i])


In [ ]:
_,ax = plt.subplots(figsize=(12,6),ncols=2)
ax[0].imshow(data[0])
ax[1].imshow(data[1])
